In [ ]:
!unzip -q /content/drive/MyDrive/FoCus_modified.zip

In [ ]:
import json
import nltk
from nltk import sent_tokenize
# nltk.download('punkt')
import time
from pprint import pprint
# from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
dataset = "/content/FoCus_modified"

In [ ]:
knowledge_df = pd.read_csv("/content/drive/MyDrive/UMBC/train_knowledge_1.csv")

In [ ]:
knowledge_df.shape

(57888, 7)

In [ ]:
for idx, data in enumerate(tqdm(test_data[:])):
  dialogID = data['dialogID']
  utterance = data['utterance']
  print(utterance)
  for i, _ in enumerate(utterance):
    knowledge_index = utterance[i]['knowledge_answer_index']
    knowledge_candidate = utterance[i]['knowledge_candidates'][knowledge_index]
    ground_personas = [utterance[i]['persona_candidate'][idx] for idx, val in enumerate(utterance[i]['persona_grounding']) if val == True ]


  0%|          | 0/1600 [00:00<?, ?it/s]

[{'dialogue1': ["I know this place, but I don't remember the name of this place.", 'This is your favorite place..The Theatre of Dionysus is an ancient Greek theatre in Athens that you would like to visit.'], 'persona_grounding': [True, True, False, False, False], 'persona_candidate': ['I like theatre.', 'I would like to go to Athens.', 'I like hiking hills.', 'I am interested in orchestra.', 'I have heard about Dionysus.'], 'knowledge_candidates': ['The ballpark was built in the Southeastern Atlanta neighborhood of Summerhill.', 'Piranhas can be bought as pets in some areas, but they are illegal in many parts of the United States, and in the Philippines, where importers face six months to four years in jail, and the piranhas destroyed to prevent proliferation in the latter.', 'The project will transform the Hangang Parks and the areas around Hangang\ninto a place of culture, art, leisure, and riverside cities.', 'The Theatre of Dionysus (or Theatre of Dionysos, gr: Θέατρο του Διονύσου)

In [ ]:
!pip install datasets --quiet

     |████████████████████████████████| 441 kB 7.0 MB/s 
     |████████████████████████████████| 95 kB 5.0 MB/s 
     |████████████████████████████████| 163 kB 49.1 MB/s 
     |████████████████████████████████| 115 kB 53.2 MB/s 
     |████████████████████████████████| 212 kB 68.0 MB/s 
     |████████████████████████████████| 127 kB 76.4 MB/s 
     |████████████████████████████████| 115 kB 69.8 MB/s 


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("json", data_files="/content/FoCus_modified/train_set.json")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-dce3e5d847c37f62/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# utterance_lengths= []
# for utterance in dataset['train']['utterance']:
#   utterance_lengths.append(len(utterance))

In [ ]:
# min(utterance_lengths), max(utterance_lengths)

(4, 6)

In [ ]:
persona_lengths= []
for personas in dataset['train']['persona']:
  persona_lengths.append(len(personas))

In [ ]:
persona_candidate_columns = ["persona1", "persona2", "persona3", "persona4", "persona5"] #persona6 for none of these
persona_grounding_columns = ["persona_grounding1", "persona_grounding2", "persona_grounding3", "persona_grounding4", "persona_grounding5"] #persona_grounding6 for none of these

In [ ]:
dataset['train']

Dataset({
    features: ['dialogID', 'persona', 'landmark_link', 'knowledge', 'utterance'],
    num_rows: 10284
})

In [ ]:
results = []
for idx, utterances in enumerate(dataset['train']['utterance']):
  for i, utterance in enumerate(utterances):
    context = {
        "dialogID":dataset['train']['dialogID'][idx],
        "utterance": i
    }
    persona_candidates = utterance['persona_candidate']
    persona_groundings = utterance['persona_grounding']

    for col, cand in zip(persona_candidate_columns, persona_candidates):
      context[col] = cand
    for col, cand in zip(persona_grounding_columns, persona_groundings):
      context[col] = cand
    results.append(context)

In [ ]:
df = pd.DataFrame.from_dict(results)

In [ ]:
df.shape, knowledge_df.shape

((57928, 12), (57888, 7))

In [ ]:
df.to_csv("train_persona.csv", index=False)

In [ ]:
df.head()

,dialogID,utterance,persona1,persona2,persona3,persona4,persona5,persona_grounding1,persona_grounding2,persona_grounding3,persona_grounding4,persona_grounding5
0,CGHGPTDOA0O7,0,I like Gothic Revival style.,I am aware of National Heritage List.,I would like to go to england.,I am not an architect.,I hate Dragon.,False,False,False,False,False
1,CGHGPTDOA0O7,1,I like Gothic Revival style.,I am aware of National Heritage List.,I would like to go to england.,I am not an architect.,I hate Dragon.,False,False,True,False,False
2,CGHGPTDOA0O7,2,I like Gothic Revival style.,I am aware of National Heritage List.,I would like to go to england.,I am not an architect.,I hate Dragon.,False,False,False,False,False
3,CGHGPTDOA0O7,3,I like Gothic Revival style.,I am aware of National Heritage List.,I would like to go to england.,I am not an architect.,I hate Dragon.,False,False,False,True,False
4,CGHGPTDOA0O7,4,I like Gothic Revival style.,I am aware of National Heritage List.,I would like to go to england.,I am not an architect.,I hate Dragon.,False,False,False,False,False


In [ ]:
combined_df = pd.merge(knowledge_df, df, how='inner', on=["dialogID", "utterance"])

In [ ]:
combined_df.shape

(57888, 17)

In [ ]:
persona_count = {}
for row in combined_df.iterrows():

  num_persona = 0
  for col in persona_grounding_columns:
    if row[1][col] == True:
      num_persona+=1
  if num_persona > 1:
    persona_count[row[0]] = num_persona

In [ ]:
persona_count

In [ ]:
# drop_df = combined_df.drop(persona_count.keys())

In [ ]:
combined_df.to_csv("train_knowledge_persona_combined.csv", index=False)

In [ ]:
!cp /content/train_persona.csv /content/drive/MyDrive/UMBC

In [ ]:
! pip install transformers

In [ ]:
import transformers

print(transformers.__version__)

4.23.1


In [ ]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [ ]:
from datasets import load_metric

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

In [ ]:
test_dataset = Dataset.from_pandas(combined_df)

In [ ]:
test_dataset

Dataset({
    features: ['dialogID', 'utterance', 'query', 'hit_knowledge', 'ground_knowledge', 'ground_persona', 'similarity_score', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona_grounding1', 'persona_grounding2', 'persona_grounding3', 'persona_grounding4', 'persona_grounding5', '__index_level_0__'],
    num_rows: 9035
})

In [ ]:
test_dataset['query'][7]

'What can be found at this site?'

In [ ]:
def preprocess_function(examples):

  first_sentences = [[f"{query} {hit_knowledge}"]*5 for query, hit_knowledge in zip(examples['query'], examples['hit_knowledge'])]
  second_sentences = [[examples[persona_candidate_column][i] for persona_candidate_column in persona_candidate_columns]for i, _ in enumerate(examples['dialogID'])]
  first_sentences = sum(first_sentences, [])
  second_sentences = sum(second_sentences, [])

  tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
  return {k: [v[i:i+5] for i in range(0, len(v), 5)] for k, v in tokenized_examples.items()}


In [ ]:
examples = test_dataset[:5]

In [ ]:
features = preprocess_function(test_dataset[0:5])
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 5 [100, 104, 101, 102, 104]


KeyError: ignored

In [ ]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] what was the capacity of the theater? the first orchestra terrace was constructed on the site around the mid - to late - sixth century bc, where it hosted the city dionysia. the theatre reached its fullest extent in the fourth century bc under the epistates of lycurgus when it would have had a capacity of up to 17, 000, and was in continuous use down to the roman period. [SEP] i like theatre. [SEP]',
 '[CLS] what was the capacity of the theater? the first orchestra terrace was constructed on the site around the mid - to late - sixth century bc, where it hosted the city dionysia. the theatre reached its fullest extent in the fourth century bc under the epistates of lycurgus when it would have had a capacity of up to 17, 000, and was in continuous use down to the roman period. [SEP] i would like to go to athens. [SEP]',
 '[CLS] what was the capacity of the theater? the first orchestra terrace was constructed on the site around the mid - to late - sixth century bc, where it hosted

In [ ]:
encoded_datasets = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
encoded_datasets

Dataset({
    features: ['dialogID', 'utterance', 'query', 'hit_knowledge', 'ground_knowledge', 'ground_persona', 'similarity_score', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona_grounding1', 'persona_grounding2', 'persona_grounding3', 'persona_grounding4', 'persona_grounding5', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9035
})

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets[i].items() if k in accepted_keys} for i in range(10)]


In [ ]:
features[0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
test_dataset.push_to_hub("kanak8278/focus_test")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!pip install huggingface_hub transformers datasets

     |████████████████████████████████| 127 kB 81.3 MB/s 
     |████████████████████████████████| 115 kB 65.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset, load_metric, Dataset

In [ ]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
dataset = load_dataset("kanak8278/focus_test")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/kanak8278___parquet/kanak8278--focus_test-abce8c13e83d11e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialogID', 'utterance', 'query', 'hit_knowledge', 'ground_knowledge', 'ground_persona', 'similarity_score', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona_grounding1', 'persona_grounding2', 'persona_grounding3', 'persona_grounding4', 'persona_grounding5', '__index_level_0__'],
        num_rows: 9035
    })
})

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,dialogID,utterance,query,hit_knowledge,ground_knowledge,ground_persona,similarity_score,persona1,persona2,persona3,persona4,persona5,persona_grounding1,persona_grounding2,persona_grounding3,persona_grounding4,persona_grounding5,__index_level_0__
0,4P3EWN6HNTMN,0,"I know this place, but I don't remember the na...",The Theatre of Dionysus (or Theatre of Dionyso...,The Theatre of Dionysus (or Theatre of Dionyso...,"['I like theatre.', 'I would like to go to Ath...",0.945484,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,True,True,False,False,False,0
1,4P3EWN6HNTMN,1,Tell me more about the location of the theatre?,The Theatre of Dionysus (or Theatre of Dionyso...,It is built on the south slope of the Akropoli...,[],0.668992,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,False,False,1
2,4P3EWN6HNTMN,2,What about the first drama shown there?,and was reorganised during the Kleisthenic ref...,The first orchestra terrace was constructed on...,['I am interested in orchestra.'],0.412627,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,True,False,2
3,4P3EWN6HNTMN,3,What was the capacity of the theater ?,The first orchestra terrace was constructed on...,The theatre reached its fullest extent in the ...,[],0.684810,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,False,False,3
4,4P3EWN6HNTMN,4,When was it fallen into decay?,The theatre then fell into decay in the Byzant...,The theatre then fell into decay in the Byzant...,[],0.849662,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,False,False,4


In [ ]:
labels = []
for idx, row in df.iterrows():
  labels.append([1 if row[col] is True else 0 for col in persona_grounding_columns ])

In [ ]:
df['label'] = labels

In [ ]:
df.head()

,dialogID,utterance,query,hit_knowledge,ground_knowledge,ground_persona,similarity_score,persona1,persona2,persona3,persona4,persona5,persona_grounding1,persona_grounding2,persona_grounding3,persona_grounding4,persona_grounding5,__index_level_0__,label
0,4P3EWN6HNTMN,0,"I know this place, but I don't remember the na...",The Theatre of Dionysus (or Theatre of Dionyso...,The Theatre of Dionysus (or Theatre of Dionyso...,"['I like theatre.', 'I would like to go to Ath...",0.945484,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,True,True,False,False,False,0,"[1, 1, 0, 0, 0]"
1,4P3EWN6HNTMN,1,Tell me more about the location of the theatre?,The Theatre of Dionysus (or Theatre of Dionyso...,It is built on the south slope of the Akropoli...,[],0.668992,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,False,False,1,"[0, 0, 0, 0, 0]"
2,4P3EWN6HNTMN,2,What about the first drama shown there?,and was reorganised during the Kleisthenic ref...,The first orchestra terrace was constructed on...,['I am interested in orchestra.'],0.412627,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,True,False,2,"[0, 0, 0, 1, 0]"
3,4P3EWN6HNTMN,3,What was the capacity of the theater ?,The first orchestra terrace was constructed on...,The theatre reached its fullest extent in the ...,[],0.684810,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,False,False,3,"[0, 0, 0, 0, 0]"
4,4P3EWN6HNTMN,4,When was it fallen into decay?,The theatre then fell into decay in the Byzant...,The theatre then fell into decay in the Byzant...,[],0.849662,I like theatre.,I would like to go to Athens.,I like hiking hills.,I am interested in orchestra.,I have heard about Dionysus.,False,False,False,False,False,4,"[0, 0, 0, 0, 0]"


In [ ]:
for idx, row in df.iterrows():
  print(type(row['label']), type(row['label'][0]))
  break

<class 'list'> <class 'int'>


In [ ]:
test_dataset = Dataset.from_pandas(df)

In [ ]:
encoded_dataset = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
encoded_dataset[0].keys()

dict_keys(['dialogID', 'utterance', 'query', 'hit_knowledge', 'ground_knowledge', 'ground_persona', 'similarity_score', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona_grounding1', 'persona_grounding2', 'persona_grounding3', 'persona_grounding4', 'persona_grounding5', '__index_level_0__', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_dataset[i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
batch['labels']

tensor([[1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0]])

In [ ]:
batch["input_ids"][7][0].tolist()

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

['[CLS] is it a large structure? the circle is located on a constructed platform of stones up to 24 metres ( 79 ft ) across and 0. 6 metres ( 2 ft 0 in ) high. on top of this there is a 15 - metre ( 49 ft ) polygonal cairn surrounded by a granite kerb of slabs and blocks. [SEP] i am interested in old stone circles. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] is it a large structure? the circle is located on a constructed platform of stones up to 24 metres ( 79 ft ) across and 0. 6 metres ( 2 ft 0 in ) high. on top of this there is a 15 - metre ( 49 ft ) polygonal cairn surrounded by a granite kerb of slabs and blocks. [SEP] i would like to visit scotland. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] is it a large structure? the c

In [ ]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint, num_labels=5)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-focus",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,
Cloning https://huggingface.co/kanak8278/bert-base-uncased-finetuned-focus into local empty directory.


In [ ]:
trainer.train()

In [ ]:
trainer.args

In [ ]:
import torch
from torch import nn

In [ ]:
loss = nn.CrossEntropyLoss()

# # Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

In [ ]:
input, target, output

(tensor([[ 0.1447,  1.3018,  1.0631,  1.4413, -0.2837],
         [ 3.0932,  0.5725,  2.0093,  0.7446,  1.3483],
         [ 0.1597,  1.5877,  0.3147, -0.1930,  0.9460]], requires_grad=True),
 tensor([[0.1481, 0.2904, 0.3386, 0.0484, 0.1745],
         [0.0712, 0.2767, 0.1988, 0.2820, 0.1713],
         [0.1335, 0.0211, 0.6199, 0.1390, 0.0865]]),
 tensor(2.0795, grad_fn=<DivBackward1>))

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [ ]:
input, target, output

(tensor([[-0.5443,  1.6846,  0.5542,  0.1395,  0.0477],
         [-0.8571, -1.5483,  2.2143,  0.3455, -0.3058],
         [-0.1297, -1.8101,  0.7935,  2.4058,  1.0943]], requires_grad=True),
 tensor([3, 1, 3]),
 tensor(2.2096, grad_fn=<NllLossBackward0>))

In [ ]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
outs = model(batch['input_ids'], token_type_ids=None, attention_mask=batch['attention_mask'])

NameError: ignored